In [1]:
suppressPackageStartupMessages({
    library(Seurat)
    library(celldex)
    library(dplyr)
    library(SingleR)
    library(openxlsx)
    library(dplyr)
})

In [ ]:
load(file=file.path("objects", "02.rda"))

In [89]:
library(Seurat)
library(openxlsx)
library(dplyr)

# Function to clean and truncate sheet names if they are too long
clean_sheetname <- function(name, max_length = 31) {
    if (nchar(name) > max_length) {
        name <- trimws(name)
        name <- gsub("[^A-Za-z0-9]", "", name)
    }
    return(name)
}

# Function to replace an existing sheet or create a new one
replace_or_add_sheet <- function(wb, sheet_name) {
  # Check if the sheet already exists
  if (sheet_name %in% names(wb)) {
    # Remove the existing sheet
    removeWorksheet(wb, sheet = sheet_name)
  }
  # Add the new sheet
  addWorksheet(wb, sheetName = sheet_name)
}

# Function to perform differential expression analysis and save results to Excel
perform_DE_analysis <- function(seurat_obj, broad_types, condition_col, split_by, output_dir) {
  # Ensure output directory exists
  if (!dir.exists(output_dir)) {
    dir.create(output_dir, recursive = TRUE)
  }
  
  # Loop over each broad immune cell type
  for (broad_type in broad_types) {
    # Create a new workbook for each broad type
    wb <- createWorkbook()
    
    # Subset the object for the current broad type
    obj_subset <- subset(seurat_obj, broad.type == broad_type)
    
    # Get unique fine types for the current broad type
    fine_types <- unique(obj_subset@meta.data$fine.type)
    
    # Split the object by fine type
    SeuratObject_splitlist <- SplitObject(obj_subset, split.by = "fine.type")
    
    # Set identity to condition for each subset
    SeuratObject_splitlist <- lapply(SeuratObject_splitlist, function(x) {
      SetIdent(x, value = x@meta.data[[condition_col]])
    })
    
    # Loop over each fine type
    for (fine_type in fine_types) {
      # Prepare and clean the sheet name
      sheetname <- gsub("^\\s*\\d+\\.\\s*", "", fine_type)
      sheetname <- clean_sheetname(sheetname)
    #   sheetname <- make.names(sheetname, unique = TRUE) # Ensure unique and valid sheet names

      if (fine_type == "7. IGHG+ IGL- Plasma") sheetname <- paste("7. IGHG+ IGL- Plasma", "2")
      
      # Replace or add the sheet for each fine type within the broad type
      replace_or_add_sheet(wb, sheetname)
      
      # Run FindMarkers for the current fine type and handle errors
      tryCatch({
        cell_markers <- FindMarkers(SeuratObject_splitlist[[fine_type]], ident.1 = 'LN', ident.2 = 'Control',
         min.pct = 0.25, logfc.threshold = 0.25)

        cell_counts <- table(Idents(SeuratObject_splitlist[[fine_type]]))

        # Extract cell counts for clusters
        num_cells_cluster_1 <- cell_counts["LN"]
        num_cells_cluster_2 <- cell_counts["Control"]

        # Add cell counts to marker results
        cell_markers$CellCount_Cluster_1 <- num_cells_cluster_1
        cell_markers$CellCount_Cluster_2 <- num_cells_cluster_2

        # Calculate the number of cells expressing each gene and round to the nearest whole number
        cell_markers$Cells_Expressing_Cluster_1 <- round(cell_markers$pct.1 * cell_markers$CellCount_Cluster_1)
        cell_markers$Cells_Expressing_Cluster_2 <- round(cell_markers$pct.2 * cell_markers$CellCount_Cluster_2)

        # Filter significant results
        # cell_markers <- cell_markers %>% filter(p_val_adj <= 0.01)
        
        if (nrow(cell_markers) == 0) {
          cell_markers <- data.frame(Message = "No significant markers found")
        }
        
        # Write the data to the corresponding sheet
        writeData(wb, sheet = sheetname, x = cell_markers, rowNames = TRUE)
        
      }, error = function(e) {
        # Handle errors by writing the error message
        error_message <- paste("Error:", e$message)
        writeData(wb, sheet = sheetname, x = data.frame(Message = error_message), rowNames = FALSE)
      })
    }
    
    # Save the workbook to a file named after the broad type
    filename <- paste("AMP_II_SLE", gsub(" ", "_", broad_type), "Markers.xlsx", sep = "_")
    saveWorkbook(wb, file = file.path(output_dir, filename), overwrite = TRUE)
  }
}

# # Example usage:
# # Set parameters
# seurat_obj <- obj # Your Seurat object
# broad_types <- c('T Cell', 'NK Cell', 'Plasma Cell', 'B Cell', 'Myeloid Cell')
# condition_col <- 'Type' # Metadata column representing conditions
# split_by <- 'Type' # Metadata column to split by
# output_dir <- "Markers/Test"

# # Call the function
# perform_DE_analysis(seurat_obj, broad_types, condition_col, split_by, output_dir)

In [91]:
# Immune cells
seurat_obj <- obj # Your Seurat object
broad_types <- c('T Cell', 'NK Cell', 'Plasma Cell', 'B Cell', 'Myeloid Cell')
condition_col <- 'Type' # Metadata column representing conditions
split_by <- 'Type' # Metadata column to split by
output_dir <- "Markers/Fine/Immune"

# Call the function
perform_DE_analysis(seurat_obj, broad_types, condition_col, split_by, output_dir)

In [90]:
# Structural cells
seurat_obj <- obj 
broad_types <- setdiff(unique(obj@meta.data$broad.type), broad_types)
condition_col <- 'Type' 
output_dir <- "Markers/Fine/Structural"

perform_DE_analysis(seurat_obj, broad_types, condition_col, split_by, output_dir)

In [92]:
library(Seurat)
library(openxlsx)
library(dplyr)

# Function to clean and truncate sheet names if they are too long
clean_sheetname <- function(name, max_length = 31) {
    if (nchar(name) > max_length) {
        name <- trimws(name)
        name <- gsub("[^A-Za-z0-9]", "", name)
    }
    return(name)
}

# Function to replace an existing sheet or create a new one
replace_or_add_sheet <- function(wb, sheet_name) {
  if (sheet_name %in% names(wb)) {
    removeWorksheet(wb, sheet = sheet_name)
  }
  addWorksheet(wb, sheetName = sheet_name)
}

# Function to perform differential expression analysis and save results to Excel
perform_DE_analysis <- function(seurat_obj, broad_types, condition_col, output_dir) {
  # Ensure output directory exists
  if (!dir.exists(output_dir)) {
    dir.create(output_dir, recursive = TRUE)
  }
  
  # Loop over each broad immune cell type
  for (broad_type in broad_types) {
    # Create a new workbook for each broad type
    wb <- createWorkbook()
    
    # Subset the object for the current broad type
    obj_subset <- subset(seurat_obj, broad.type == broad_type)
    
    # Set identity to the condition
    SeuratObject_splitlist <- list(broad_type = obj_subset)
    SeuratObject_splitlist <- lapply(SeuratObject_splitlist, function(x) {
      SetIdent(x, value = x@meta.data[[condition_col]])
    })
    
    # Prepare and clean the sheet name
    sheetname <- clean_sheetname(broad_type)
    
    # Replace or add the sheet for the broad type
    replace_or_add_sheet(wb, sheetname)
    
    # Run FindMarkers for the current broad type and handle errors
    tryCatch({
      cell_markers <- FindMarkers(SeuratObject_splitlist[[broad_type]], ident.1 = 'LN', ident.2 = 'Control',
       min.pct = 0.25, logfc.threshold = 0.25)

      cell_counts <- table(Idents(SeuratObject_splitlist[[broad_type]]))

      # Extract cell counts for clusters
      num_cells_cluster_1 <- cell_counts["LN"]
      num_cells_cluster_2 <- cell_counts["Control"]

      # Add cell counts to marker results
      cell_markers$CellCount_Cluster_1 <- num_cells_cluster_1
      cell_markers$CellCount_Cluster_2 <- num_cells_cluster_2

      # Calculate the number of cells expressing each gene and round to the nearest whole number
      cell_markers$Cells_Expressing_Cluster_1 <- round(cell_markers$pct.1 * cell_markers$CellCount_Cluster_1)
      cell_markers$Cells_Expressing_Cluster_2 <- round(cell_markers$pct.2 * cell_markers$CellCount_Cluster_2)

      # Filter significant results
      # cell_markers <- cell_markers %>% filter(p_val_adj <= 0.01)

      if (nrow(cell_markers) == 0) {
        cell_markers <- data.frame(Message = "No significant markers found")
      }
      
      # Write the data to the corresponding sheet
      writeData(wb, sheet = sheetname, x = cell_markers, rowNames = TRUE)
      
    }, error = function(e) {
      # Handle errors by writing the error message
      error_message <- paste("Error:", e$message)
      writeData(wb, sheet = sheetname, x = data.frame(Message = error_message), rowNames = FALSE)
    })
    
    # Save the workbook to a file named after the broad type
    filename <- paste("AMP_II_SLE", gsub(" ", "_", broad_type), "Markers.xlsx", sep = "_")
    saveWorkbook(wb, file = file.path(output_dir, filename), overwrite = TRUE)
  }
}

# Example usage:
# seurat_obj <- obj # Your Seurat object
# broad_types <- c('T Cell', 'NK Cell', 'Plasma Cell', 'B Cell', 'Myeloid Cell')
# condition_col <- 'Type' # Metadata column representing conditions
# output_dir <- "Markers/Test"

# perform_DE_analysis(seurat_obj, broad_types, condition_col, output_dir)


In [ ]:
# Immune cells
seurat_obj <- obj # Your Seurat object
broad_types <- c('T Cell', 'NK Cell', 'Plasma Cell', 'B Cell', 'Myeloid Cell')
condition_col <- 'Type' # Metadata column representing conditions
split_by <- 'Type' # Metadata column to split by
output_dir <- "Markers/Fine/Immune"

# Call the function
perform_DE_analysis(seurat_obj, broad_types, condition_col, split_by, output_dir)